In [1]:
import bson
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook
from skimage.data import imread
import io

In [4]:
categories = pd.read_csv('category_names.csv')

os.mkdir('train')
os.mkdir('test')
    
for category in tqdm_notebook(categories.index):
    os.mkdir(os.path.join('train', str(category)))
    
os.mkdir('checkpoints')
os.mkdir('predict')

a3 = ['a3', 'a3_n1', 'a3_n6', 'a3_x1', 'a3_x2']
v2 = ['v2', 'v2_n1', 'v2_x1', 'v2_x2']

for model in a3:
    os.mkdir(os.path.join('checkpoints', model))
    os.mkdir(os.path.join('predict', model))
    
for model in v2:
    os.mkdir(os.path.join('checkpoints', model))
    os.mkdir(os.path.join('predict', model))
    

A Jupyter Widget

In [ ]:
num_products = 7069896  # 7069896 for train and 1768182 for test
out_folder = 'train'

bar = tqdm_notebook(total=num_products)

train = []
train_imgs = []
with open('train.bson', 'rb') as fbson:

    data = bson.decode_file_iter(fbson)
    
    for c, d in enumerate(data):
        category = d['category_id']
        _id = d['_id']
        for e, pic in enumerate(d['imgs']):
            fname = os.path.join(str(category), '{}-{}.jpg'.format(_id, e))
            
            with open(fname, 'wb') as f:
                f.write(pic['picture'])
            
            train.append([fname, _id, category, np.where(categories.category_id.values==category)[0][0]])

        train_imgs.append([_id, category, len(d['imgs'])])
        
        bar.update()

In [ ]:
num_products = 1768182
bar = tqdm_notebook(total=num_products)

out_folder = 'test'
test_imgs = []

with open('test.bson', 'rb') as fbson:

    data = bson.decode_file_iter(fbson)
    
    for c, d in enumerate(data):
        _id = d['_id']
        for e, pic in enumerate(d['imgs']):
            fname = '{}-{}.jpg'.format(_id, e)
            
            with open(fname, 'wb') as f:
                f.write(pic['picture'])
                
        test_imgs.append([_id, len(d['imgs'])])

        bar.update()

In [ ]:
np.random.shuffle(np.array(train))
df = pd.DataFrame(train)
df.to_csv('train2.csv')

df2 = pd.DataFrame(train_imgs, columns=['_id', 'category_id', 'n_images'])
df2.to_csv('train_imgs.csv')

for i in range(len(df2)):
    label = np.where(categories.category_id.values==df2.loc[i, 'category_id'])[0][0]
    df2.set_value(i,'y', label)

df2.y = df2.y.astype('int64')

df4 = df2.sample(frac=1).reset_index(drop=True)
df4[['_id', 'category_id', 'n_images', 'y']].to_csv('train4.csv', index=False)

df3 = pd.DataFrame(test_imgs, columns=['_id', 'n_images'])
df3.to_csv('test_imgs.csv')